In [169]:
import urllib.request
import re
from bs4 import BeautifulSoup
import sqlite3
import time
import math
import pandas
x = input('輸入X軸座標:')
y = input('輸入X軸座標:')
page = 1
coordinate_x = []
coordinate_y = []
crops_type = []
crop_oases = []
distance = []
spend_time = []

def distance_caculate(x1,x2,y1,y2):
    x1 = math.fabs(x1)
    x2 = math.fabs(x2)
    y1 = math.fabs(y1)
    y2 = math.fabs(y2)
    dis = round(pow(math.sqrt(math.fabs(x1 - x2)) + math.sqrt(math.fabs(y1 - y2)),0.5),4)
    return dis

def time_caculate(speed,unit):
    #speed=7, 4 grids spend 00:17:09
    total_distance = 257.25 * unit
    total_second = int(round(total_distance * float(7/speed),0))
    total_time = datetime.timedelta(seconds=total_second)
    return total_time

#start crawler
for i in range(1,10):
    
    if page <= 1:
        url = 'http://croppers.travibot.com/?l=en&server=tx2.travian.com&x={}&y={}'.format(x,y)
    else:
        url = 'http://croppers.travibot.com/?l=en&server=tx2.travian.com&x={}&y={}&page={}'.format(x,y,page)
    print ("{:%H:%M:%S}".format(datetime.datetime.now()) + " start search: {}".format(url))
    response = urllib.request.urlopen(url)
    html = response.read()
    sp = BeautifulSoup(html.decode('utf8'),"html.parser")
    temp1 = sp.find_all("td",class_="l")
    temp2 = sp.find_all("a",href=re.compile("http://tx2.travian.com/karte.php?"))

    for i in temp1:
        if i.find_all("img",alt=re.compile("Cropper type")):
            crops_type.append(i.get_text())
        elif i.find_all("img",src=re.compile("/img/4.gif")):
            crop_oases.append('Yes')
        else:
            crop_oases.append('No')

    for i in temp2:
        coordinate_x.append(i.get_text().split('|')[0])
        coordinate_y.append(i.get_text().split('|')[1])
    
    page+=1
    
    #aviod web server overloading
    time.sleep(3)

for i in range(1,len(coordinate_x)):
    distance.append(distance_caculate(float(coordinate_x[i-1]),float(x),float(coordinate_y[i-1]),float(y)))
    spend_time.append(str(time_caculate(5,distance[i-1])))
                                                                                            
#show = () 
#for i in range(1,len(coordinate_x)):
#    show = (coordinate_x[i-1],coordinate_y[i-1],crops_type[i-1],crop_oases[i-1],distance[i-1],spend_time[i-1])
#    print(show)

show = ()
tmp = []
for i in range(1,len(coordinate_x)):
    show = (coordinate_x[i-1],coordinate_y[i-1],crops_type[i-1],crop_oases[i-1],str(distance[i-1]),str(spend_time[i-1]))
    print(show)
    tmp.append(show)
    my_df = pandas.DataFrame(tmp,columns=['X-axis', 'Y-axis', 'crops_type', 'crop_oases', 'distance', 'spend_time'])
    my_df.to_csv(r'C:\Users\Administrator\Desktop\Travian.csv', index=False, header=True)
    
connect_db = sqlite3.connect('travian.db')
c = connect_db.cursor()

try:
    c.execute('''DROP TABLE crops_info''')
except:
    print('Table already exists.')
    
c.execute('''CREATE TABLE crops_info (coordinate_x integer, coordinate_y integer, crops_type integer\t
        , crop_oases text, distance real, spend_time text)''')

for i in range(1,len(coordinate_x)):
    add_info = (coordinate_x[i-1],coordinate_y[i-1],crops_type[i-1],crop_oases[i-1],distance[i-1],spend_time[i-1])
    c.execute('INSERT INTO crops_info VALUES (?,?,?,?,?,?)', add_info)

c.execute('''SELECT * FROM crops_info ORDER BY distance ASC''')

with open(r'C:\Users\Administrator\Desktop\Travian.sql', 'w') as f:
    for line in connect_db.iterdump():
        f.write('%s\n' % line)

connect_db.commit()
connect_db.close()

輸入X軸座標:1
輸入X軸座標:1
16:04:24 start search: http://croppers.travibot.com/?l=en&server=tx2.travian.com&x=1&y=1
16:04:29 start search: http://croppers.travibot.com/?l=en&server=tx2.travian.com&x=1&y=1&page=2
16:04:33 start search: http://croppers.travibot.com/?l=en&server=tx2.travian.com&x=1&y=1&page=3
16:04:38 start search: http://croppers.travibot.com/?l=en&server=tx2.travian.com&x=1&y=1&page=4
16:04:42 start search: http://croppers.travibot.com/?l=en&server=tx2.travian.com&x=1&y=1&page=5
16:04:47 start search: http://croppers.travibot.com/?l=en&server=tx2.travian.com&x=1&y=1&page=6
16:04:52 start search: http://croppers.travibot.com/?l=en&server=tx2.travian.com&x=1&y=1&page=7
16:04:56 start search: http://croppers.travibot.com/?l=en&server=tx2.travian.com&x=1&y=1&page=8
16:05:01 start search: http://croppers.travibot.com/?l=en&server=tx2.travian.com&x=1&y=1&page=9
('0', '0', ' 9', 'No', '1.4142', '0:08:29')
('2', '3', ' 9', 'No', '1.5538', '0:09:20')
('2', '5', ' 15', 'Yes', '1.7321', '0